# EDA - HDB Resale

## Model Development - Regression 

### Import Environment Modules and Data

In [4]:
### Import Python Modules

## System Modules
import sys
import os
import warnings
import logging
import time
import random
from typing import Union, Optional


## Essential Modules
import numpy as np
import pandas as pd


## Graphical Modules
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

## Statistical Tests
import scipy.stats as st
from scipy.stats import randint
#from scipy.stats import median_abs_deviation
from scipy.stats.mstats import kruskalwallis

import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
#from statsmodels.stats.weightstats import DescrStatsW
from statsmodels.tsa.seasonal import STL                 # trend‑seasonality split  :contentReference[oaicite:0]{index=0}
from statsmodels.tsa.stattools import adfuller           # stationarity test      :contentReference[oaicite:1]{index=1}
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf  # autocorr lenses   :contentReference[oaicite:2]{index=2}


## Pipeline and Train Test Split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold
from sklearn.compose import ColumnTransformer

## SciKit Learning Preprocessing  
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, PowerTransformer
from sklearn.preprocessing import OneHotEncoder

### SciKit Learn ML Models

## Linear Models
from sklearn.linear_model import LinearRegression, HuberRegressor, QuantileRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.compose import TransformedTargetRegressor

## Tree Based Linear Models
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

## Performance Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score


### Hyper-parameter Tuning

## GridSearch CV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score

## explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV # type: ignore

## explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingRandomSearchCV # type: ignore

## Optuna for Hyperparameter Tuning
from optuna.integration import OptunaSearchCV
from optuna.distributions import IntDistribution, FloatDistribution


## Problem Statement

**We work for a property consultancy company. This company want to develop an end to end machine learning pipeline that could deliver housing price prediction to customer.**

**Our task is to develop a machine learning model that could accurately predict the resale prices of HDB resale flats. This model will assist buyers or sellers in planning their budgets more effectively and set realistic expectations. This model also need to help buyers determine the type of flat they can afford and in which location. This model also should provide sellers with valuable information regarding the potential market value of their property.**

## Data Preprocessing For Model Building

In [5]:
# Load the dataset
df = pd.read_csv('./data/hdb_resale_price.csv')

In [6]:
len(df)

211086

### Removing Duplicates

In [7]:
# Remove duplicated items
df.drop_duplicates(inplace=True)

In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
len(df)

210782

In [10]:
df.describe()

,floor_area_sqm,lease_commence_date,resale_price
count,210782.000000,210782.000000,2.107820e+05
mean,96.860812,1996.336770,5.176226e+05
std,24.038202,14.241823,1.828416e+05
min,31.000000,1966.000000,1.400000e+05
25%,82.000000,1985.000000,3.800000e+05
50%,93.000000,1996.000000,4.850000e+05
75%,112.000000,2011.000000,6.200000e+05
max,366.700000,2022.000000,1.658888e+06


## Feature Engineering

### Feature Engineering - Splitting Transaction Column 'month' to Year and Month

In [11]:
def convert_month_to_year_month(df: pd.DataFrame, 
                              month: str = 'month',
                              transaction_year: str = 'transaction_year',
                              transaction_month: str = 'transaction_month') -> pd.DataFrame:
    """
    Convert month column to separate year and month columns.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe containing the month column
    month : str, default 'month'
        Name of the column containing month data
    transaction_year : str, default 'transaction_year'
        Name of the output year column
    month_out_col : str, default 'transaction_month'
        Name of the output month column
    
    Returns:
    --------
    pd.DataFrame
        Modified dataframe
    
    Raises:
    -------
    ValueError
        If month_col doesn't exist in the dataframe
    TypeError
        If df is not a pandas DataFrame
    """
    
    # Input validation
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame")
    
    # Check if dataframe is empty
    if df.empty:
        warnings.warn("Input dataframe is empty", UserWarning)
        return df.copy()
    
    # Check if month column exists
    if month not in df.columns:
        raise ValueError(f"Column '{month}' not found in dataframe")
    
    # Work on copy
    df = df.copy()
    
    # Convert to datetime with error handling
    try:
        df[month] = pd.to_datetime(df[month], format='%Y-%m', errors='coerce')
    except (ValueError, TypeError) as e:
        raise ValueError(f"Cannot convert column '{month}' to datetime: {str(e)}")
    
    # Extract year and month
    df[transaction_year] = df[month].dt.year
    df[transaction_month] = df[month].dt.month
    
    # Return dataframe
    return df

In [12]:
# =============================================================================
# SIMPLE TESTS - Run these in your notebook
# =============================================================================

def test_basic_functionality():
    """Test that the function works with normal data"""
    # Create test data
    test_df = pd.DataFrame({
        'month': ['2023-01', '2023-02', '2023-03'],
        'value': [100, 200, 300]
    })
    
    # Run function
    result = convert_month_to_year_month(test_df)
    
    # Check results
    assert 'transaction_year' in result.columns
    assert 'transaction_month' in result.columns
    assert result['transaction_year'].tolist() == [2023, 2023, 2023]
    assert result['transaction_month'].tolist() == [1, 2, 3]
    
    print("✓ Basic functionality test passed!")


test_basic_functionality()

✓ Basic functionality test passed!


In [13]:
def test_different_date_formats():
    """Test with different date formats"""
    test_df = pd.DataFrame({
        'month': ['2023-01', '2023-02', '2023-03'],
        'value': [100, 200, 300]
    })
    
    result = convert_month_to_year_month(test_df)
    
    assert result['transaction_year'].iloc[0] == 2023
    assert result['transaction_month'].iloc[0] == 1
    
    print("✓ Different date formats test passed!")

test_different_date_formats()

✓ Different date formats test passed!


In [14]:
def test_with_nulls():
    """Test with null values"""
    test_df = pd.DataFrame({
        'month': ['2023-01', None, '2023-03'],
        'value': [100, 200, 300]
    })
    
    result = convert_month_to_year_month(test_df)
    
    # First and third rows should work
    assert result['transaction_year'].iloc[0] == 2023
    assert result['transaction_month'].iloc[0] == 1
    
    print("✓ Null values test passed!")

test_with_nulls()

✓ Null values test passed!


In [15]:
def test_original_unchanged():
    """Test that original dataframe is not modified"""
    original_df = pd.DataFrame({
        'month': ['2023-01', '2023-02'],
        'value': [100, 200]
    })
    
    # Store original state
    original_columns = original_df.columns.tolist()
    
    # Run function
    result = convert_month_to_year_month(original_df)
    
    # Check original is unchanged
    assert original_df.columns.tolist() == original_columns
    assert 'transaction_year' not in original_df.columns
    
    # Check result has new columns
    assert 'transaction_year' in result.columns
    
    print("✓ Original unchanged test passed!")

test_original_unchanged()

✓ Original unchanged test passed!


In [16]:
# Splitting column 'month' into year and month
df = convert_month_to_year_month(df)

In [17]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,transaction_year,transaction_month
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017,1
1,2017-01-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017,1
2,2017-01-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017,1
3,2017-01-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,2017,1
4,2017-01-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,2017,1


### Feature Engineering - Convert 'remaining_lease' to remaining_lease_months'

In [18]:
def convert_lease_to_month(lease):
    """
    Convert remaining lease period from string to total number of months.
    Args:
        remaining_lease in (str)

    Returns: 
        integer

    Example:
        convert_lease_to_month('07 TO 09') -> 8.0  
    """
    str_list = lease.split(' ')
    if ('months' in str_list) | ('month' in str_list):
        year = int(str_list[0])
        month = int(str_list[2])
        t_month = (year * 12) + month 
    elif ('years' in str_list) & (('months' not in str_list) | ('month' not in str_list)):
        year = int(str_list[0])
        t_month = (year * 12)
    else:
        year = int(str_list[0])
        t_month = (year * 12)        
    return t_month

In [19]:
def test_convert_lease_to_month_simple():
    """Simple test cases for convert_lease_to_month function"""
    
    # Test years and months format
    assert convert_lease_to_month('5 years 6 months') == 66  # 5*12 + 6 = 66
    assert convert_lease_to_month('2 years 3 months') == 27  # 2*12 + 3 = 27
    assert convert_lease_to_month('1 years 0 months') == 12  # 1*12 + 0 = 12
    
    # Test years only format
    assert convert_lease_to_month('10 years') == 120  # 10*12 = 120
    assert convert_lease_to_month('5 years') == 60    # 5*12 = 60
    assert convert_lease_to_month('1 years') == 12    # 1*12 = 12
    
    # Test with singular 'month'
    assert convert_lease_to_month('3 years 1 month') == 37   # 3*12 + 1 = 37
    assert convert_lease_to_month('0 years 1 month') == 1    # 0*12 + 1 = 1
    
    # Test just numbers (should default to years)
    assert convert_lease_to_month('5') == 60    # 5*12 = 60
    assert convert_lease_to_month('2') == 24    # 2*12 = 24
    print("All simple test cases passed!")

test_convert_lease_to_month_simple()

All simple test cases passed!


In [20]:
def test_convert_lease_to_month_variations():
    """Test various input formats"""
    
    # Test different plural/singular combinations
    assert convert_lease_to_month('1 year 1 month') == 13    # Should work if 'year' -> 'years'
    assert convert_lease_to_month('10 years 11 months') == 131  # 10*12 + 11 = 131
    
    # Test zero cases
    assert convert_lease_to_month('0 years 6 months') == 6   # 0*12 + 6 = 6
    assert convert_lease_to_month('0 years') == 0      
    print("All variations test cases passed!")

test_convert_lease_to_month_variations()

All variations test cases passed!


In [21]:
# Convert column remaining lease to remaining lease by  months
df['remaining_lease_by_months'] = df.remaining_lease.apply(convert_lease_to_month)


In [22]:

df[['remaining_lease', 'remaining_lease_by_months']].sample(10)

,remaining_lease,remaining_lease_by_months
182360,53 years 08 months,644
95389,46 years 10 months,562
151225,60 years 03 months,723
132431,80 years 01 month,961
202191,57 years 05 months,689
204002,62 years 11 months,755
171330,58 years 05 months,701
94807,55 years 05 months,665
107717,75 years 09 months,909
117887,55 years 03 months,663


### Feature Engineering - 'storey_range'

In [23]:
def convert_storey_range(storey_range):
    """
    Convert storey range to the numerical average.
    Args:
        storey_range in (str)

    Returns: 
        float

    Example:
        convert_storey_range('07 TO 09') -> 8.0 
    """

    low, high = storey_range.split(' TO ')
    average = (int(low) + int(high)) / 2
    return average

In [24]:
def test_convert_storey_range():
    """Test cases for convert_storey_range function"""
    
    # Basic test case from the example
    assert convert_storey_range('07 TO 09') == 8.0
    
    # Test with single digits
    assert convert_storey_range('1 TO 3') == 2.0
    assert convert_storey_range('5 TO 7') == 6.0
    
    # Test with same storey (no range)
    assert convert_storey_range('05 TO 05') == 5.0
    assert convert_storey_range('10 TO 10') == 10.0
    
    # Test with larger ranges
    assert convert_storey_range('01 TO 05') == 3.0
    assert convert_storey_range('10 TO 20') == 15.0
    
    # Test with double digits
    assert convert_storey_range('12 TO 16') == 14.0
    assert convert_storey_range('25 TO 35') == 30.0
    
    # Test with leading zeros
    assert convert_storey_range('01 TO 03') == 2.0
    assert convert_storey_range('08 TO 12') == 10.0
    
    # Test odd ranges (result should be .5)
    assert convert_storey_range('1 TO 2') == 1.5
    assert convert_storey_range('10 TO 11') == 10.5
    
    # Test with higher floors
    assert convert_storey_range('50 TO 60') == 55.0
    assert convert_storey_range('99 TO 101') == 100.0

    print("All test cases passed!")


test_convert_storey_range()


All test cases passed!


In [25]:
def test_convert_storey_range_edge_cases():
    """Test edge cases and potential error scenarios"""
    
    # Test ground floor scenarios
    assert convert_storey_range('0 TO 2') == 1.0
    assert convert_storey_range('00 TO 01') == 0.5
    
    # Test with mixed formatting
    assert convert_storey_range('5 TO 15') == 10.0
    assert convert_storey_range('02 TO 8') == 5.0

    print("All edge case tests passed!")

test_convert_storey_range_edge_cases()

All edge case tests passed!


In [26]:
import pytest
def test_convert_storey_range_errors():
    """Test error handling scenarios"""
    
    # Test invalid format (should raise ValueError)
    with pytest.raises(ValueError):
        convert_storey_range('invalid format')
    
    with pytest.raises(ValueError):
        convert_storey_range('5-7')  # Wrong separator
    
    with pytest.raises(ValueError):
        convert_storey_range('5 TO')  # Missing high value
    
    with pytest.raises(ValueError):
        convert_storey_range('TO 7')  # Missing low value
    
    # Test non-numeric values
    with pytest.raises(ValueError):
        convert_storey_range('A TO B')
    
    with pytest.raises(ValueError):
        convert_storey_range('1 TO B')

    print("All error handling tests passed!")

test_convert_storey_range_errors()

All error handling tests passed!


In [27]:
# Alternative test format using unittest if you prefer
import unittest

class TestConvertStoreyRange(unittest.TestCase):
    
    def test_basic_functionality(self):
        """Test basic functionality"""
        self.assertEqual(convert_storey_range('07 TO 09'), 8.0)
        self.assertEqual(convert_storey_range('1 TO 3'), 2.0)
        self.assertEqual(convert_storey_range('10 TO 20'), 15.0)
        print("Basic functionality test passed!")
    
    def test_same_storey(self):
        """Test when low and high are the same"""
        self.assertEqual(convert_storey_range('05 TO 05'), 5.0)
        self.assertEqual(convert_storey_range('10 TO 10'), 10.0)
        print("Same storey test passed!")
    
    def test_decimal_results(self):
        """Test cases that result in decimal values"""
        self.assertEqual(convert_storey_range('1 TO 2'), 1.5)
        self.assertEqual(convert_storey_range('10 TO 11'), 10.5)
        print("Decimal results test passed!")
    
    def test_invalid_input(self):
        """Test invalid input handling"""
        with self.assertRaises(ValueError):
            convert_storey_range('invalid')
        with self.assertRaises(ValueError):
            convert_storey_range('A TO B')
        print("Invalid input test passed!")


unit_test = TestConvertStoreyRange()
unit_test.test_basic_functionality()
unit_test.test_same_storey()
unit_test.test_decimal_results()
unit_test.test_invalid_input()

Basic functionality test passed!
Same storey test passed!
Decimal results test passed!
Invalid input test passed!


In [28]:
# convert storey range to number middle value
df['storey_range'] = df.storey_range.apply(convert_storey_range)

In [29]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,transaction_year,transaction_month,remaining_lease_by_months
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,11.0,44.0,Improved,1979,61 years 04 months,232000.0,2017,1,736
1,2017-01-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,2.0,67.0,New Generation,1978,60 years 07 months,250000.0,2017,1,727
2,2017-01-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,2.0,67.0,New Generation,1980,62 years 05 months,262000.0,2017,1,749
3,2017-01-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,5.0,68.0,New Generation,1980,62 years 01 month,265000.0,2017,1,745
4,2017-01-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,2.0,67.0,New Generation,1980,62 years 05 months,265000.0,2017,1,749


### Dropping Irrelevant Columns

In [30]:
# Dropping irrelevant columns for machine learning preparation
# Keep month for temporal sorting and split, will remove later
irrelevant_columns = ['block', 'street_name', 'remaining_lease', 'lease_commence_date']
df.drop(columns = irrelevant_columns, inplace=True)

In [31]:
df.head()

,month,town,flat_type,storey_range,floor_area_sqm,flat_model,resale_price,transaction_year,transaction_month,remaining_lease_by_months
0,2017-01-01,ANG MO KIO,2 ROOM,11.0,44.0,Improved,232000.0,2017,1,736
1,2017-01-01,ANG MO KIO,3 ROOM,2.0,67.0,New Generation,250000.0,2017,1,727
2,2017-01-01,ANG MO KIO,3 ROOM,2.0,67.0,New Generation,262000.0,2017,1,749
3,2017-01-01,ANG MO KIO,3 ROOM,5.0,68.0,New Generation,265000.0,2017,1,745
4,2017-01-01,ANG MO KIO,3 ROOM,2.0,67.0,New Generation,265000.0,2017,1,749


In [32]:
df.columns

Index(['month', 'town', 'flat_type', 'storey_range', 'floor_area_sqm',
       'flat_model', 'resale_price', 'transaction_year', 'transaction_month',
       'remaining_lease_by_months'],
      dtype='object')

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 210782 entries, 0 to 211085
Data columns (total 10 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   month                      210782 non-null  datetime64[ns]
 1   town                       210782 non-null  object        
 2   flat_type                  210782 non-null  object        
 3   storey_range               210782 non-null  float64       
 4   floor_area_sqm             210782 non-null  float64       
 5   flat_model                 210782 non-null  object        
 6   resale_price               210782 non-null  float64       
 7   transaction_year           210782 non-null  int32         
 8   transaction_month          210782 non-null  int32         
 9   remaining_lease_by_months  210782 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int32(2), int64(1), object(3)
memory usage: 16.1+ MB


## Data Split

### Temporal Split

In [34]:
X_temporal = df.set_index('month').sort_index().copy() 
y_temporal = X_temporal['resale_price']
X_temporal = X_temporal.drop(columns=['resale_price'])

In [35]:
X_temporal.head()

,town,flat_type,storey_range,floor_area_sqm,flat_model,transaction_year,transaction_month,remaining_lease_by_months
month,,,,,,,,
2017-01-01,ANG MO KIO,2 ROOM,11.0,44.0,Improved,2017,1,736
2017-01-01,SEMBAWANG,4 ROOM,2.0,100.0,Model A,2017,1,981
2017-01-01,SEMBAWANG,4 ROOM,5.0,100.0,Model A,2017,1,976
2017-01-01,SEMBAWANG,4 ROOM,2.0,86.0,Model A2,2017,1,990
2017-01-01,QUEENSTOWN,5 ROOM,20.0,110.0,Improved,2017,1,1067


In [36]:
y_temporal.head()

month
2017-01-01    232000.0
2017-01-01    342000.0
2017-01-01    335000.0
2017-01-01    305000.0
2017-01-01    860000.0
Name: resale_price, dtype: float64

In [37]:
print("X_temporal shape:", X_temporal.shape)
print("y_temporal shape:", y_temporal.shape)

X_temporal shape: (210782, 8)
y_temporal shape: (210782,)


In [38]:
# find split date for training and validation
split_date_train = df['month'].quantile(0.6)
split_date_train

Timestamp('2022-05-01 00:00:00')

In [39]:
split_date_validation = df['month'].quantile(0.8) 
split_date_validation

Timestamp('2023-12-01 00:00:00')

In [40]:
# Split the data into training and testing sets in a temporal manner
X_train_temporal = X_temporal[X_temporal.index < split_date_train]
y_train_temporal = y_temporal[y_temporal.index < split_date_train]
X_validation_temporal = X_temporal[(X_temporal.index >= split_date_train) & (X_temporal.index < split_date_validation)]
y_validation_temporal = y_temporal[(y_temporal.index >= split_date_train) & (y_temporal.index < split_date_validation)]
X_test_temporal = X_temporal[X_temporal.index >= split_date_validation]
y_test_temporal = y_temporal[y_temporal.index >= split_date_validation]


In [41]:
# Display the shapes of the splits to verify
print("Temporal Training set shape:", X_train_temporal.shape, y_train_temporal.shape)
print("Temporal Validation set shape:", X_validation_temporal.shape, y_validation_temporal.shape)
print("Temporal Test set shape:", X_test_temporal.shape, y_test_temporal.shape)

Temporal Training set shape: (125284, 8) (125284,)
Temporal Validation set shape: (41584, 8) (41584,)
Temporal Test set shape: (43914, 8) (43914,)


### Data Split Random

In [42]:
df.drop(columns=['month'], inplace=True)

In [43]:
# Separate the data into features and target
X = df.drop(columns='resale_price').copy()
y = df['resale_price']

In [44]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (210782, 8)
y shape: (210782,)


In [45]:
X.head()

,town,flat_type,storey_range,floor_area_sqm,flat_model,transaction_year,transaction_month,remaining_lease_by_months
0,ANG MO KIO,2 ROOM,11.0,44.0,Improved,2017,1,736
1,ANG MO KIO,3 ROOM,2.0,67.0,New Generation,2017,1,727
2,ANG MO KIO,3 ROOM,2.0,67.0,New Generation,2017,1,749
3,ANG MO KIO,3 ROOM,5.0,68.0,New Generation,2017,1,745
4,ANG MO KIO,3 ROOM,2.0,67.0,New Generation,2017,1,749


In [46]:
y.head()

0    232000.0
1    250000.0
2    262000.0
3    265000.0
4    265000.0
Name: resale_price, dtype: float64

In [47]:
# Split the data into training (60%) and test-validation (40%) sets
X_train, X_validation_and_test, y_train, y_validation_and_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Split the test-validation set (40%) into validation (20%) and test (20%) sets
X_validation, X_test, y_validation, y_test = train_test_split(X_validation_and_test, y_validation_and_test, test_size=0.5, random_state=42)

In [48]:
# Display the shapes of the splits to verify
print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_validation.shape, y_validation.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Training set shape: (126469, 8) (126469,)
Validation set shape: (42156, 8) (42156,)
Test set shape: (42157, 8) (42157,)


## Setting Feature Scaling

In [49]:
X_train.head()

,town,flat_type,storey_range,floor_area_sqm,flat_model,transaction_year,transaction_month,remaining_lease_by_months
8447,CHOA CHU KANG,4 ROOM,8.0,104.0,Model A,2017,6,850
64263,ANG MO KIO,3 ROOM,5.0,82.0,New Generation,2020,1,710
144387,ANG MO KIO,3 ROOM,11.0,81.0,New Generation,2023,12,667
19535,JURONG EAST,3 ROOM,5.0,86.0,Improved,2017,12,725
157883,PUNGGOL,5 ROOM,14.0,114.0,Premium Apartment,2023,6,960


In [50]:
numerical_features = X_train.select_dtypes(include='number').columns.tolist()
categorical_columns = X_train.select_dtypes(include=['object']).columns.tolist()
temporal_columns = X_train.select_dtypes(include=['datetime']).columns.tolist()


In [51]:
numerical_features

['storey_range',
 'floor_area_sqm',
 'transaction_year',
 'transaction_month',
 'remaining_lease_by_months']

In [52]:
categorical_columns

['town', 'flat_type', 'flat_model']

In [53]:
temporal_columns

[]

In [54]:
# Select columns that are numerical for feature scaling preparation
numerical_features = ['floor_area_sqm', 'remaining_lease_by_months', 'transac_year'] 

degree = 1  # Degree of polynomial features, can be adjusted
# Create a numerical transformer pipeline
numerical_transformer = Pipeline(steps=[
    ('polynomial_features', PolynomialFeatures(degree=degree)),  # Placeholder for polynomial features
    ('scaler', StandardScaler())
])

## Feature Encoding

In [55]:
# Select columns that need to be one-hot encoded
nominal_features = ['transac_month', 'town', 'flat_model', 'flat_type']

# Select the columns that do not required further processing 
passthrough_features = ['storey_range']

# Setting pipeline for one-hot encoding
nominal_transformer = Pipeline(steps=[
    ('one_hot', OneHotEncoder(handle_unknown='ignore'))
])


## Preprocessor

In [56]:
# print numerical, nominal and passthrough features
print("Numerical features:", numerical_features)
print("Nominal features:", nominal_features)    
print("Passthrough features:", passthrough_features)

Numerical features: ['floor_area_sqm', 'remaining_lease_by_months', 'transac_year']
Nominal features: ['transac_month', 'town', 'flat_model', 'flat_type']
Passthrough features: ['storey_range']


In [57]:
degree = 1  # Degree of polynomial features, can be adjusted
# Create a numerical transformer pipeline
numerical_transformer = Pipeline(steps=[
    ('polynomial_features', PolynomialFeatures(degree=degree)),  # Placeholder for polynomial features
    ('scaler', StandardScaler())
])

In [58]:
# Setting pipeline for one-hot encoding
nominal_transformer = Pipeline(steps=[
    ('one_hot', OneHotEncoder(handle_unknown='ignore'))
])

In [59]:
# Setting preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('nom', nominal_transformer, nominal_features),
        ('pass', 'passthrough', passthrough_features) 
    ],
    remainder='passthrough',
    n_jobs=-1
    )

In [60]:
preprocessor

ColumnTransformer(n_jobs=-1, remainder='passthrough',
                  transformers=[('num',
                                 Pipeline(steps=[('polynomial_features',
                                                  PolynomialFeatures(degree=1)),
                                                 ('scaler', StandardScaler())]),
                                 ['floor_area_sqm', 'remaining_lease_by_months',
                                  'transac_year']),
                                ('nom',
                                 Pipeline(steps=[('one_hot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['transac_month', 'town', 'flat_model',
                                  'flat_type']),
                                ('pass', 'passthrough', ['storey_range'])])

## Model Development - Regression

### Multivariate Linear Regression without Target Transformation

In [ ]:
# Setting up regression pipeline
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [ ]:
# Model fitting
lr_pipeline.fit(X_train, y_train)

In [ ]:
# Prediction on validation set
y_val_pred = lr_pipeline.predict(X_val)
y_val_pred

In [ ]:
# Get feature names after preprocessing
feature_names = lr_pipeline.named_steps['preprocessor'].get_feature_names_out()
print(feature_names)

In [ ]:
# get model coefficients
model_coefficients = lr_pipeline.named_steps['regressor'].coef_
print(model_coefficients)

In [ ]:
# Calculate regression metrics for validation set
lr_val_mae = mean_absolute_error(y_val, y_val_pred)
lr_val_mse = mean_squared_error(y_val, y_val_pred)
lr_val_rmse = root_mean_squared_error(y_val, y_val_pred)  
lr_val_r2 = r2_score(y_val, y_val_pred)

# Display the metrics
print('Linear Regression Performance Metrics:')
print(f"Linear Regression Validation MAE: {lr_val_mae}")
print(f"Linear Regression Validation MSE: {lr_val_mse}")
print(f"Linear Regression Validation RMSE: {lr_val_rmse}")
print(f"Linear Regression Validation R2: {lr_val_r2}")

### Analyzing Residual Plot

In [ ]:
# Residual analysis
residuals = y_val - y_val_pred[0]
# Plotting residuals
plt.figure(figsize=(10, 6))
plt.scatter(y_val_pred, residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# residual distribution
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=30)
plt.title('Residuals Distribution')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

**Residual plot shows that the model did not capture non-linearity in the target. Residual distribution remains skewed. Will try target transformation.**

### Multivariate Linear Regression with Target Transformation

In [ ]:
pt = PowerTransformer(method='yeo-johnson')
y_train_transformed = pt.fit_transform(y_train.values.reshape(-1, 1))

In [ ]:
# Plotting the transformed target variable
plt.figure(figsize=(10, 6))
sns.histplot(y_train_transformed, kde=True, bins=30)
plt.title('Transformed Target Variable Distribution')
plt.xlabel('Transformed Resale Price')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Model fitting
lr_pipeline.fit(X_train, y_train_transformed)

In [ ]:
# Prediction on validation set
y_val_pred_transformed = lr_pipeline.predict(X_val)
#print(type(y_val_pred_transformed))
#print(y_val_pred_transformed.shape)
y_val_pred = pt.inverse_transform(y_val_pred_transformed.reshape(-1, 1))  # type: ignore # Inverse transform to get original scale

In [ ]:
# Calculate regression metrics for validation set
lr_val_mae_transformed = mean_absolute_error(y_val, y_val_pred)
lr_val_mse_transformed = mean_squared_error(y_val, y_val_pred)
lr_val_rmse_transformed = root_mean_squared_error(y_val, y_val_pred)  
lr_val_r2_transformed = r2_score(y_val, y_val_pred)


# Display the metrics
print(f"Validation MAE (Transformed): {lr_val_mae_transformed}")
print(f"Validation MSE (Transformed): {lr_val_mse_transformed}")
print(f"Validation RMSE (Transformed): {lr_val_rmse_transformed}")
print(f"Validation R2 (Transformed): {lr_val_r2_transformed}")

In [ ]:
# Residual analysis
residuals = y_val - y_val_pred[0]

# Plotting residuals
plt.figure(figsize=(10, 6))
plt.scatter(y_val_pred, residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Predicted Values (Transformed)')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# Residual distribution
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=30)
plt.title('Residuals Distribution (Transformed)')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# residual plot in transform space
y_pred_t = lr_pipeline.predict(X_val)          
if isinstance(y_pred_t, tuple):
	y_pred_t_flat = y_pred_t[0].flatten()
else:
	y_pred_t_flat = y_pred_t.flatten()
resid_t = y_train_transformed.flatten()[:len(y_pred_t_flat)] - y_pred_t_flat
plt.scatter(y_pred_t_flat, resid_t, alpha=0.3)
plt.axhline(0, color='red', linestyle='--')
plt.title("Residuals vs Predicted (Transformed Space)")
plt.show()


**Comparing the residual plot after target transformation, we can see that linear regression could not catch non-linearity.**

In [ ]:
# Display the metrics
print(f"Validation MAE: {lr_val_mae}")
print(f"Validation MSE: {lr_val_mse}")
print(f"Validation RMSE: {lr_val_rmse}")
print(f"Validation R2: {lr_val_r2}")

# Display the metrics
print(f"Validation MAE (Transformed): {lr_val_mae_transformed}")
print(f"Validation MSE (Transformed): {lr_val_mse_transformed}")
print(f"Validation RMSE (Transformed): {lr_val_rmse_transformed}")
print(f"Validation R2 (Transformed): {lr_val_r2_transformed}")

The transformation likely helped normalize the error distribution and reduce the impact of large errors (outliers) in the transformed space. This makes the model more accurate on average, leading to a better Mean Absolute Error.

RMSE got worst and there is a slight dip in R-squared due to the inverse transformation. While the model might be performing well in the transformed scale, any prediction errors, particularly those on the higher end of the original data, get magnified when inverse transformed back to the original scale. Since RMSE heavily penalizes these larger errors (by squaring them), it increases. This overall increase in magnified errors also leads to a slight decrease in R-squared, as the model explains less of the variance in the original scale.

Essentially, the transformation optimized for model performance in a normalized space, but the re-scaling back to the original units amplified certain errors, negatively impacting RMSE and R-squared.

**Transformed target did decrease MAE, which is our business objective, residual plot shows that the model failed to capture non-linearity. We will test with Polynomial regression, Ridge and Lasso regression. Will also try Huber Regression and Quantile Regression.**

### Polynomial Regression with/without Transformation

In [ ]:
degree = 2  # Degree of polynomial features, can be adjusted
# Create a numerical transformer pipeline
numerical_transformer = Pipeline(steps=[
    ('polynomial_features', PolynomialFeatures(degree=degree)),  # Placeholder for polynomial features
    ('scaler', StandardScaler())
])

In [ ]:
# Setting preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('nom', nominal_transformer, nominal_features),
        ('pass', 'passthrough', passthrough_features) 
    ],
    remainder='passthrough',
    n_jobs=-1
    )

In [ ]:
# Setting up regression pipeline
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

#### Polynomial Regression (No Transformation)

In [ ]:
lr_pipeline.fit(X_train, y_train)

In [ ]:
y_val_pred = lr_pipeline.predict(X_val)

In [ ]:
X_val[0,:]

In [ ]:
# Calculate regression metrics for validation set
poly_val_mae = mean_absolute_error(y_val, y_val_pred)
poly_val_mse = mean_squared_error(y_val, y_val_pred)
poly_val_rmse = root_mean_squared_error(y_val, y_val_pred)  
poly_val_r2 = r2_score(y_val, y_val_pred)

# Display the metrics
print(f"Validation MAE: {poly_val_mae}")
print(f"Validation MSE: {poly_val_mse}")
print(f"Validation RMSE: {poly_val_rmse}")
print(f"Validation R2: {poly_val_r2}")

#### Polynomial Regression (Transformed Target Regressor)

In [ ]:
TransformedTargetRegressor_model = TransformedTargetRegressor(
    regressor    = lr_pipeline,
    transformer  = PowerTransformer(method='yeo-johnson')
)
TransformedTargetRegressor_model.fit(X_train, y_train)
y_val_pred = TransformedTargetRegressor_model.predict(X_val)

In [ ]:
# Calculate regression metrics for validation set
poly_val_mae_transformed = mean_absolute_error(y_val, y_val_pred)
poly_val_mse_transformed = mean_squared_error(y_val, y_val_pred)
poly_val_rmse_transformed = root_mean_squared_error(y_val, y_val_pred)  
poly_val_r2_transformed = r2_score(y_val, y_val_pred)

# Display the metrics
print(f"Polynomial Regression Validation MAE with {degree} polynomial degree: {poly_val_mae}")
print(f"Polynomial Regression Validation MSE with {degree} polynomial degree: {poly_val_mse}")
print(f"Polynomial Regression Validation RMSE with {degree} polynomial degree: {poly_val_rmse}")
print(f"Polynomial Regression Validation R2 with {degree} polynomial degree: {poly_val_r2}")

# Display the metrics
print(f"Polynomial Regression Validation MAE (Transformed)  with {degree} polynomial degree: {poly_val_mae_transformed}")
print(f"Polynomial Regression Validation MSE (Transformed)  with {degree} polynomial degree: {poly_val_mse_transformed}")
print(f"Polynomial Regression Validation RMSE (Transformed)  with {degree} polynomial degree: {poly_val_rmse_transformed}")
print(f"Polynomial Regression Validation R2 (Transformed)  with {degree} polynomial degree: {poly_val_r2_transformed}")



**The results shows that target transformation in polynomial regression did improve the performance slightly.** 

### Ridge Regression

In [ ]:
# Setting pipeline for Ridge Regression 
ridge_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge(random_state=42))
])


ridge_pipeline.fit(X_train, y_train)
y_val_pred_ridge = ridge_pipeline.predict(X_val)

In [ ]:
# Calculate regression metrics for validation set with Ridge Regression
val_mae_ridge = mean_absolute_error(y_val, y_val_pred_ridge)
val_mse_ridge = mean_squared_error(y_val, y_val_pred_ridge)
val_rmse_ridge = root_mean_squared_error(y_val, y_val_pred_ridge)  
val_r2_ridge = r2_score(y_val, y_val_pred_ridge)

# Print the metrics for Ridge Regression
print("Ridge Regression Metrics:")
print(f"Ridge Regression polynomial degree:{degree} Validation MAE: {val_mae_ridge}")
print(f"Ridge Regression polynomial degree:{degree} MSE: {val_mse_ridge}")
print(f"Ridge Regression polynomial degree:{degree} RMSE: {val_rmse_ridge}")
print(f"Ridge Regression polynomial degree:{degree} R²: {val_r2_ridge}")

In [ ]:
# Target Transformation for Ridge 
TransformedTargetRegressor_model = TransformedTargetRegressor(
    regressor    = ridge_pipeline,
    transformer  = PowerTransformer(method='yeo-johnson')
)
TransformedTargetRegressor_model.fit(X_train, y_train)
y_val_pred_ridge_transformed = TransformedTargetRegressor_model.predict(X_val)

In [ ]:

# Calculate regression metrics for validation set with Ridge Regression
val_mae_ridge_transformed = mean_absolute_error(y_val, y_val_pred_ridge_transformed)
val_mse_ridge_transformed = mean_squared_error(y_val, y_val_pred_ridge_transformed)
val_rmse_ridge_transformed = root_mean_squared_error(y_val, y_val_pred_ridge_transformed)  
val_r2_ridge_transformed = r2_score(y_val, y_val_pred_ridge_transformed)

# Print the metrics for Ridge Regression
print("Ridge Regression Metrics:")
print(f"Ridge Regression polynomial degree:{degree} Target Transformation Validation MAE: {val_mae_ridge_transformed}")
print(f"Ridge Regression polynomial degree:{degree} Target Transformation MSE: {val_mse_ridge_transformed}")
print(f"Ridge Regression polynomial degree:{degree} Target Transformation RMSE: {val_rmse_ridge_transformed}")
print(f"Ridge Regression polynomial degree:{degree} Target Transformation R²: {val_r2_ridge_transformed}")

### Lasso Regression

In [ ]:
from sklearn.exceptions import ConvergenceWarning
ConvergenceWarning('ignore')

# Fit Lasso Regression with default alpha
lasso_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Lasso(alpha = 0.001, max_iter=3000, random_state=42))
])

TransformedTargetRegressor_model = TransformedTargetRegressor(
    regressor    = lasso_pipeline,
    transformer  = PowerTransformer(method='yeo-johnson')
)
TransformedTargetRegressor_model.fit(X_train, y_train)
y_val_pred_lasso = TransformedTargetRegressor_model.predict(X_val)

# Calculate regression metrics for validation set with Lasso Regression
val_mae_lasso_transformed = mean_absolute_error(y_val, y_val_pred_lasso)
val_mse_lasso_transformed = mean_squared_error(y_val, y_val_pred_lasso)
val_rmse_lasso_transformed = root_mean_squared_error(y_val, y_val_pred_lasso)  # RMSE is the square root of MSE
val_r2_lasso_transformed = r2_score(y_val, y_val_pred_lasso)

# Display the metrics for Lasso Regression
print("Lasso Regression Metrics:")
print(f"Lasso Transformed Validation MAE: {val_mae_lasso_transformed}")
print(f"Lasso Transformed Validation MSE: {val_mse_lasso_transformed}")
print(f"Lasso Transformed Validation RMSE: {val_rmse_lasso_transformed}")
print(f"Lasso Transformed Validation R²: {val_r2_lasso_transformed}")

In [ ]:
from sklearn.exceptions import ConvergenceWarning
ConvergenceWarning('ignore')

# Fit Lasso Regression with default alpha
lasso_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Lasso(alpha=0.001, max_iter=3000, random_state=42))
])

lasso_pipeline.fit(X_train, y_train)
y_val_pred_lasso = lasso_pipeline.predict(X_val)

# Calculate regression metrics for validation set with Lasso Regression
val_mae_lasso = mean_absolute_error(y_val, y_val_pred_lasso)
val_mse_lasso = mean_squared_error(y_val, y_val_pred_lasso)
val_rmse_lasso = root_mean_squared_error(y_val, y_val_pred_lasso)  # RMSE is the square root of MSE
val_r2_lasso = r2_score(y_val, y_val_pred_lasso)

# Display the metrics for Lasso Regression
print("Lasso Regression Metrics:")
print(f"Lasso Validation MAE: {val_mae_lasso}")
print(f"Lasso Validation MSE: {val_mse_lasso}")
print(f"Lasso Validation RMSE: {val_rmse_lasso}")
print(f"Lasso Validation R²: {val_r2_lasso}")

### Huber Regression

This is an experiment on Huber Regression as no transformation is required.

In [ ]:
degree = 2  # Degree of polynomial features, can be adjusted
# Create a numerical transformer pipeline
numerical_transformer = Pipeline(steps=[
    ('polynomial_features', PolynomialFeatures(degree=degree)),  # Placeholder for polynomial features
    ('scaler', StandardScaler())
])

In [ ]:
# Setting preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('nom', nominal_transformer, nominal_features),
        ('pass', 'passthrough', passthrough_features) 
    ],
    remainder='passthrough',
    n_jobs=-1
    )

In [ ]:
# Setting pipeline for Huber Regression 
huber_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', HuberRegressor(epsilon=1.35, max_iter=3000, alpha=0.001)) # usually 1.35 epsilon
])


huber_pipeline.fit(X_train, y_train)
y_val_pred_huber = huber_pipeline.predict(X_val)

In [ ]:
# Calculate regression metrics for validation set with Huber Regression
val_mae_huber = mean_absolute_error(y_val, y_val_pred_huber)
val_mse_huber = mean_squared_error(y_val, y_val_pred_huber)
val_rmse_huber = root_mean_squared_error(y_val, y_val_pred_huber)  
val_r2_huber = r2_score(y_val, y_val_pred_huber)

# Display the metrics for Huber Regression
print("Huber Regression Metrics:")
print(f"Huber Validation MAE: {val_mae_huber}")
print(f"Huber Validation MSE: {val_mse_huber}")
print(f"Huber Validation RMSE: {val_rmse_huber}")
print(f"Huber Validation R²: {val_r2_huber}")

### Quantile Regression

This is an experiment on Quantile regression to check the performance against traditional linear models.

In [ ]:
len(X_train)

In [ ]:
# Perform train test split
X_train_q, _, y_train_q, _ = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

In [ ]:
# Setting pipeline for Quantile Regression (mid quantile)
quantile_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', QuantileRegressor(quantile=0.5))
])

quantile_pipeline.fit(X_train, y_train)
y_val_pred_quant_5 = quantile_pipeline.predict(X_val)

In [ ]:
# Calculate regression metrics for validation set with Quantile Regression
val_mae_quant_5 = mean_absolute_error(y_val, y_val_pred_quant_5)
val_mse_quant_5 = mean_squared_error(y_val, y_val_pred_quant_5)
val_rmse_quant_5 = root_mean_squared_error(y_val, y_val_pred_quant_5)  
val_r2_quant_5 = r2_score(y_val, y_val_pred_quant_5)

# Display the metrics for Quantile Regression
print("Quantile Regression Metrics:")
print(f"Quantile Validation MAE: {val_mae_quant_5}")
print(f"Quantile Validation MSE: {val_mse_quant_5}")
print(f"Quantile Validation RMSE: {val_rmse_quant_5}")
print(f"Quantile Validation R²: {val_r2_quant_5}")

In [ ]:
# Setting pipeline for Quantile Regression (top end quantile)
quantile_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', QuantileRegressor(quantile=0.9))
])

quantile_pipeline.fit(X_train, y_train)
y_val_pred_quant_9 = quantile_pipeline.predict(X_val)

In [ ]:
# Calculate regression metrics for validation set with Quantile Regression
val_mae_quant_9 = mean_absolute_error(y_val, y_val_pred_quant_9)
val_mse_quant_9 = mean_squared_error(y_val, y_val_pred_quant_9)
val_rmse_quant_9 = root_mean_squared_error(y_val, y_val_pred_quant_9)  
val_r2_quant_9 = r2_score(y_val, y_val_pred_quant_9)

# Display the metrics for Quantile Regression
print("Quantile Regression Metrics:")
print(f"Quantile Validation MAE: {val_mae_quant_9}")
print(f"Quantile Validation MSE: {val_mse_quant_9}")
print(f"Quantile Validation RMSE: {val_rmse_quant_9}")
print(f"Quantile Validation R²: {val_r2_quant_9}")

**Quantile and Huber Regression did not outperform traditional linear model. We will compare metric on Polynomial, Ridge and Lasso Regression.**

#### Linear Model Metric Analysis

In [ ]:
# Add metrics to dictionary for each model with category
metrics_poly = {
    "Category": "Original",
    "Model": "Polynomial Regression",
    "MAE":  poly_val_mae,
    "RMSE": poly_val_rmse,
    "R2":   poly_val_r2,
}

metrics_poly_transformed = {
    "Category": "Target Transformed",
    "Model": "Polynomial Regression",
    "MAE":  poly_val_mae_transformed,
    "RMSE": poly_val_rmse_transformed,
    "R2":   poly_val_r2_transformed,
}

metrics_ridge = {
    "Category": "Original",
    "Model": "Ridge Regression",
    "MAE":  val_mae_ridge,
    "RMSE": val_rmse_ridge,
    "R2":   val_r2_ridge,
}

metrics_ridge_transformed = {
    "Category": "Target Transformed",
    "Model": "Ridge Regression",
    "MAE":  val_mae_ridge_transformed,
    "RMSE": val_rmse_ridge_transformed,
    "R2":   val_r2_ridge_transformed,
}

metrics_lasso = {
    "Category": "Original",
    "Model": "Lasso Regression",
    "MAE":  val_mae_lasso,
    "RMSE": val_rmse_lasso,
    "R2":   val_r2_lasso,
}

metrics_lasso_transformed = {
    "Category": "Target Transformed",
    "Model": "Lasso Regression",
    "MAE":  val_mae_lasso_transformed,
    "RMSE": val_rmse_lasso_transformed,
    "R2":   val_r2_lasso_transformed,
}

# Append all metrics dictionaries to the results list
all_results = [
    metrics_poly,
    metrics_ridge,
    metrics_lasso,
    metrics_poly_transformed,
    metrics_ridge_transformed,
    metrics_lasso_transformed
]

# Create DataFrame with multi-level index
results_df = (
    pd.DataFrame(all_results)
      .set_index(["Category", "Model"])
      .round(4)
)

print("="*80)
print("MODEL PERFORMANCE COMPARISON")
print("="*80)
print(results_df.to_string())

# Alternative: Create separate sections with cleaner display
print("\n" + "="*80)
print("MODEL PERFORMANCE - SEPARATED BY TRANSFORMATION")
print("="*80)

# Group by category and display each section
for category in results_df.index.get_level_values('Category').unique():
    print(f"\n{category.upper()} MODELS:")
    print("-" * 80)
    section_df = results_df.loc[category].round(4)
    print(section_df.to_string())

**Regularization will reduce validation performance, but the difference is not that huge if we are looking at R2 or MAE. We need further fine tuning and apply the model to the test set before we can decide who model to use. However, we will try Decision Tree model since our target are non-linear.** 

### Decision Tree Based Baseline Model

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('nom', nominal_transformer, nominal_features),
        ('pass', 'passthrough', passthrough_features) 
    ],
    remainder='passthrough',
    n_jobs=-1
    )

In [ ]:
# Decision Tree Regressor

dt_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=42))
])
dt_pipeline.fit(X_train, y_train)   

In [ ]:
# Predict on the validation set with Decision Tree Regressor
y_val_pred_dt = dt_pipeline.predict(X_val)  
# Calculate regression metrics for validation set with Decision Tree Regressor
val_mae_dt = mean_absolute_error(y_val, y_val_pred_dt)
val_mse_dt = mean_squared_error(y_val, y_val_pred_dt)
val_rmse_dt = root_mean_squared_error(y_val, y_val_pred_dt)
val_r2_dt = r2_score(y_val, y_val_pred_dt)  

# Display the metrics for Decision Tree Regressor
print("Decision Tree Regressor Metrics:")
print(f"Decision Tree Validation MAE: {val_mae_dt}")
print(f"Decision Tree Validation MSE: {val_mse_dt}")
print(f"Decision Tree Validation RMSE: {val_rmse_dt}")
print(f"Decision Tree Validation R²: {val_r2_dt}")

In [ ]:
# Display the feature importance
dt_feature_importances = dt_pipeline.named_steps['regressor'].feature_importances_
dt_feature_names = preprocessor.get_feature_names_out()
dt_feature_importances_df = pd.DataFrame({'Feature': dt_feature_names, 'Importance': dt_feature_importances})
dt_feature_importances_df = dt_feature_importances_df.sort_values(by='Importance', ascending=False)

# Print the feature importances
print("\nDecision Tree Feature Importances (Top 20):")
print(dt_feature_importances_df[:20])  # Display top 20 features

**Using basic decision tree model without any parameter tuning, our error reduced and performance improve is much better than Linear Regression with transformed target. We have a MAE of $33K and R-squared of 92%. We believe that any parameter fine tuning on linear model will not be able to outperform the decision tree base line model. We will stop using linear models and adopt tree-based model as our recommended model.**
 

### Random Forest

In [ ]:
# Random Forest Regressor
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))
])
rf_pipeline.fit(X_train, y_train)

In [ ]:
# predict on the validation set with Random Forest Regressor
y_val_pred_rf = rf_pipeline.predict(X_val)

# Calculate regression metrics for validation set with Random Forest Regressor
val_mae_rf = mean_absolute_error(y_val, y_val_pred_rf)
val_mse_rf = mean_squared_error(y_val, y_val_pred_rf)
val_rmse_rf = root_mean_squared_error(y_val, y_val_pred_rf)
val_r2_rf = r2_score(y_val, y_val_pred_rf)

# Display the metrics for Random Forest Regressor
print("Random Forest Regressor Metrics:")
print(f"Random Forest Validation MAE: {val_mae_rf}")
print(f"Random Forest Validation MSE: {val_mse_rf}")
print(f"Random Forest Validation RMSE: {val_rmse_rf}")
print(f"Random Forest Validation R²: {val_r2_rf}")

In [ ]:
# Display the feature importances for Random Forest
rf_feature_importances = rf_pipeline.named_steps['regressor'].feature_importances_
rf_feature_names = preprocessor.get_feature_names_out()
rf_feature_importances_df = pd.DataFrame({'Feature': rf_feature_names, 'Importance': rf_feature_importances})
rf_feature_importances_df = rf_feature_importances_df.sort_values(by='Importance', ascending=False)
# Print the feature importances for Random Forest
print("\nRandom Forest Feature Importances (Top 20):")
print(rf_feature_importances_df[:20])  # Display top 20 features

### XGBoost

In [ ]:
# XGBoost Regressor
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor(random_state=42, n_jobs=-1, verbosity=0))
])
xgb_pipeline.fit(X_train, y_train)  

In [ ]:
# predict on the validation set with XGBoost Regressor
y_val_pred_xgb = xgb_pipeline.predict(X_val)

# Calculate regression metrics for validation set with XGBoost Regressor
val_mae_xgb = mean_absolute_error(y_val, y_val_pred_xgb)
val_mse_xgb = mean_squared_error(y_val, y_val_pred_xgb)
val_rmse_xgb = root_mean_squared_error(y_val, y_val_pred_xgb)
val_r2_xgb = r2_score(y_val, y_val_pred_xgb)

# Display the metrics for XGBoost Regressor
print("XGBoost Regressor Metrics:")
print(f"XGBoost Validation MAE: {val_mae_xgb}")
print(f"XGBoost Validation MSE: {val_mse_xgb}")
print(f"XGBoost Validation RMSE: {val_rmse_xgb}")
print(f"XGBoost Validation R²: {val_r2_xgb}")

In [ ]:
# Display the feature importances of XGBoost
xgb_feature_importances = xgb_pipeline.named_steps['regressor'].feature_importances_
xgb_feature_names = preprocessor.get_feature_names_out()
xgb_feature_importances_df = pd.DataFrame({'Feature': xgb_feature_names, 'Importance': xgb_feature_importances})
xgb_feature_importances_df = xgb_feature_importances_df.sort_values(by='Importance', ascending=False)

# Print the feature importances of XGBoost
print("\nXGBoost Feature Importances (Top 20):")
print(xgb_feature_importances_df[:20])  # Display top 20 features   

### LightGBM

In [ ]:
# LightGBM Regressor
lgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', lgb.LGBMRegressor(random_state=42, n_jobs=-1, verbosity=-1))
])
lgb_pipeline.fit(X_train, y_train)

In [ ]:
# predict on the validation set with LightGBM Regressor
y_val_pred_lgb = lgb_pipeline.predict(X_val)

# Calculate regression metrics for validation set with LightGBM Regressor
val_mae_lgb = mean_absolute_error(y_val, y_val_pred_lgb)
val_mse_lgb = mean_squared_error(y_val, y_val_pred_lgb)
val_rmse_lgb = root_mean_squared_error(y_val, y_val_pred_lgb)
val_r2_lgb = r2_score(y_val, y_val_pred_lgb)

# Display the metrics for LightGBM Regressor
print("LightGBM Regressor Metrics:")
print(f"LightGBM Validation MAE: {val_mae_lgb}")
print(f"LightGBM Validation MSE: {val_mse_lgb}")
print(f"LightGBM Validation RMSE: {val_rmse_lgb}")
print(f"LightGBM Validation R²: {val_r2_lgb}")

In [ ]:
# Display the feature importances of LightGBM
lgb_feature_importances = lgb_pipeline.named_steps['regressor'].feature_importances_
lgb_feature_names = preprocessor.get_feature_names_out()
lgb_feature_importances_df = pd.DataFrame({'Feature': lgb_feature_names, 'Importance': lgb_feature_importances})
lgb_feature_importances_df = lgb_feature_importances_df.sort_values(by='Importance', ascending=False)
# Print the feature importances of LightGBM
print("\nLightGBM Feature Importances (Top 20):")
print(lgb_feature_importances_df[:20])  # Display top 20 features

In [ ]:
# add ing metrics to dictionary for each model
metrics_dt = {
    "Model": "Decision Tree",
    "MAE":  val_mae_dt,
    "RMSE": val_rmse_dt,
    "R2":   val_r2_dt,
}

metrics_lgb = {
    "Model": "LightGBM",
    "MAE":  val_mae_lgb,
    "RMSE": val_rmse_lgb,
    "R2":   val_r2_lgb,
}

metrics_rf = {
    "Model": "Random Forest",
    "MAE":  val_mae_rf,
    "RMSE": val_rmse_rf,
    "R2":   val_r2_rf,
}

metrics_xgb = {
    "Model": "XGBoost",
    "MAE":  val_mae_xgb,
    "RMSE": val_rmse_xgb,
    "R2":   val_r2_xgb,
}


all_results = [metrics_dt, metrics_rf, metrics_xgb, metrics_lgb]

results_df = (
    pd.DataFrame(all_results)
      .set_index("Model")
      .round(4)          # nice, tidy formatting
)

In [ ]:
display(results_df)

In [ ]:
# Saved for final sanity check
dt_default_model = dt_pipeline
rf_default_model = rf_pipeline

**Without any parameters tuning, Random Forest shows the most promising results, followed by XGBoost and LightGBM. We will use the 3 models for further hyperparameter tuning. Decision tree without tuning will be the base line model for sanity check.** 

## Performance Metrics

In [ ]:
display(results_df)

**MSE is too difficult to interpret and thus we have dropped it from the analysis. We will be using MAE, RMSE and R-squared. We should look at all the different performance metrics MAE, RMSE and R2 because different performance metric present different information. MAE measure the magnitude of errors but it is less sensitive to outliers. RMSE penalized large errors and helps to identify if you have large outliers. R2 tell us how much of the variation in the target are explainable by our model. However, it cannot tell us if the predictions are bias.**

**Our analysis above shows that RMSE and larger than MAE indicating that there are outliers that RMSE amplified. We will not use RMSE as we do not want to penalized the squared error since the outliers are the high end housing market that our property firm would want to served. Our R-squared are consistently high indicating that our tree-based models can explain the variation in the target better. However, the differences between difference R-squared could not help us to explain the metrics further. In conclusion, we would use MAE as our primary metrics as it is also easier to explained to the management of the property firm.**   

## Hyperparameter Tuning

**Grid Search Strategy**

We will use **MAE** as the main metric because it is most easy to be understood by the stakeholder.

We have performed fine tuning with a 3 stage parameters fine tuning starting with **Randomized Search** for stage 1 using the widest search space. Then we will use **Halving Randomized Search** based on the search result of stage 1 and finally, we will use **Optuna Search** to finalized the search parameters.

However, after several hours of fine tuning, our MAE did improved a few hundred dollars. This is not acceptable as the improvement is marginal compared to the resource we have put in. Thus we will be using randomized search  with ranges around the default as sanity check.

For further fine tuning, we will be using optuna with 5 cross validation folder around our searched parameters.

### Random Search CV Parameters Settings

In [ ]:
# Define RansomizedSearchCV parameters with wide search space for Decision Tree Regressor, Random Forest Regressor, XGBoost Regressor, and LightGBM Regressor

rf_param_grid = {
    'regressor__n_estimators':    [100, 200, 300],       
    'regressor__max_depth':       [40, 50, 60],           
    'regressor__min_samples_split': [2, 5, 7, 10],
    'regressor__min_samples_leaf':  [1, 5, 7, 10],
    'regressor__max_features':      ['sqrt', 'log2', 0.5, 1.0]
}   

xgb_param_grid = {
    'regressor__n_estimators': [50, 100, 200],      
    'regressor__max_depth': [3, 5, 7, 9],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__subsample': [0.6, 0.8, 1.0],
    'regressor__colsample_bytree': [0.6, 0.8, 1.0],
    'regressor__gamma': [0, 0.1, 0.2],
    'regressor__reg_alpha': [0, 0.1, 1],
    'regressor__reg_lambda': [0, 0.1, 1]
}   

lgb_param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [3, 5, 7, 9],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__num_leaves': [31, 63, 127],
    'regressor__min_child_samples': [20, 30, 40],
    'regressor__subsample': [0.6, 0.8, 1.0],
    'regressor__colsample_bytree': [0.6, 0.8, 1.0],
    'regressor__reg_alpha': [0, 0.1, 1],
    'regressor__reg_lambda': [0, 0.1, 1]
}


In [ ]:
rf_random_search = RandomizedSearchCV(
    rf_pipeline,
    param_distributions=rf_param_grid,
    scoring='neg_mean_absolute_error',
    n_iter=5,
    cv=3,
    verbose=3,
    random_state=42,
    n_jobs=-1
)
rf_random_search.fit(X_train, y_train)

In [ ]:
# perform randomized search for XGBoost Regressor
xgb_random_search = RandomizedSearchCV(
    xgb_pipeline,
    param_distributions=xgb_param_grid,
    n_iter=60,  # Number of iterations for random search
    scoring='neg_mean_absolute_error',   
    cv=3,  # 3-fold cross-validation
    verbose=3,
    random_state=42,
    n_jobs=-1  # Use all available cores
)
xgb_random_search.fit(X_train, y_train) 

In [ ]:
# perform randomized search for LightGBM Regressor
lgb_random_search = RandomizedSearchCV(
    lgb_pipeline,
    param_distributions=lgb_param_grid,
    n_iter=30,  # Number of iterations for random search
    scoring='neg_mean_absolute_error',       
    cv=3,  # 3-fold cross-validation
    verbose=3,
    random_state=42,
    n_jobs=-1  # Use all available cores
)
lgb_random_search.fit(X_train, y_train)

In [ ]:
# Print the best parameters and best score for Random Forest Regressor
print("Best parameters for Random Forest Regressor:", rf_random_search.best_params_)
print("Best score for Random Forest Regressor (negative MSE):", rf_random_search.best_score_)
# Print the best parameters and best score for XGBoost Regressor
print("Best parameters for XGBoost Regressor:", xgb_random_search.best_params_)
print("Best score for XGBoost Regressor (negative MSE):", xgb_random_search.best_score_)
# Print the best parameters and best score for LightGBM Regressor
print("Best parameters for LightGBM Regressor:", lgb_random_search.best_params_)
print("Best score for LightGBM Regressor (negative MSE):", lgb_random_search.best_score_)

**The difference between the best model is not that great. Will perform fine tuning and select the best few for final model evaluation test.**

### Optuna Grid Search CV

In [ ]:
def make_tight_distributions(best_params,
                             int_frac: float = 0.2,
                             float_frac: float = 0.1,
                             min_int_step: int = 1):
    """
    Given a dict of best_params_, return a dict of
    Optuna Distributions that span ±frac around each value.
    """
    tight_dists = {}
    for name, val in best_params.items():
        # only handle numeric params
        if isinstance(val, int):
            # window = max(val * int_frac, min_int_step)
            window = max(int(val * int_frac), min_int_step)
            low  = max(1, val - window)      # avoid zero or negative
            high = val + window
            # choose step = min_int_step or window itself
            step = min_int_step if min_int_step <= window else window
            tight_dists[name] = IntDistribution(low=low, high=high, step=step)

        elif isinstance(val, float):
            window = val * float_frac
            low  = max(0.0, val - window)
            high = min(1.0, val + window)    # assuming [0,1] support for fractions
            tight_dists[name] = FloatDistribution(low=low, high=high)

        else:
            # skip non-numeric (e.g. categorical) or handle separately
            continue

    return tight_dists


#### Random Forest Optuna Search CV

In [ ]:

# 1. grab your previously-found best params:
best_rf_random_search = rf_random_search.best_params_

# 2. build the “around-the-best” distributions:
rf_param_distributions = make_tight_distributions(best_rf_random_search,
                                               int_frac=0.2,     # ±20%
                                               float_frac=0.1,   # ±10%
                                               min_int_step=1)   # at least step=1


In [ ]:
rf_param_distributions

In [ ]:
# 3. Optuna quick search
optuna_search = OptunaSearchCV(
    estimator=rf_pipeline,
    param_distributions=rf_param_distributions,
    cv=5,             
    n_trials=2,      
    scoring='neg_mean_absolute_error',
    random_state=42,
    n_jobs=-1,
    verbose=3,
)

# 4. Run the search
optuna_search.fit(X_train, y_train)


In [ ]:
# 5. Inspect results
print("Best MAE  =", optuna_search.best_score_)
print("Best params:")
for k, v in optuna_search.best_params_.items():
    print(f"  • {k} = {v}")

# 6. Your final model
final_rf_model = optuna_search.best_estimator_
final_rf_model_params = optuna_search.best_params_
final_rf_model_scores = optuna_search.best_score_

In [ ]:
val_final_rf_model_pred = final_rf_model.predict(X_val)

# 7. Calculate regression metrics for validation set with Random Forest Regressor
final_val_mae_rf = mean_absolute_error(y_val, val_final_rf_model_pred)
final_val_mse_rf = mean_squared_error(y_val, val_final_rf_model_pred)
final_val_rmse_rf = root_mean_squared_error(y_val, val_final_rf_model_pred)
final_val_r2_rf = r2_score(y_val, val_final_rf_model_pred)

# 8. Display the metrics for Random Forest Regressor
print("Random Forest Regressor Metrics:")
print(f"Random Forest Validation MAE: {final_val_mae_rf}")
print(f"Random Forest Validation MSE: {final_val_mse_rf}")
print(f"Random Forest Validation RMSE: {final_val_rmse_rf}")
print(f"Random Forest Validation R²: {final_val_r2_rf}")

#### XGBoost Optuna Search CV

In [ ]:
# 1. grab your previously-found best params:
best_xgb_random_search_param = xgb_random_search.best_params_

# 2. build the “around-the-best” distributions:
xgb_param_distributions = make_tight_distributions(best_xgb_random_search_param,
                                               int_frac=0.2,     # ±20%
                                               float_frac=0.1,   # ±10%
                                               min_int_step=1)   # at least step=1


In [ ]:
xgb_param_distributions

In [ ]:
# 3. Optuna quick search
optuna_search = OptunaSearchCV(
    estimator=xgb_pipeline,
    param_distributions=xgb_param_distributions,
    cv=5,             
    n_trials=50,      
    scoring='neg_mean_absolute_error',
    random_state=42,
    n_jobs=-1,
    verbose=3,
)

# 4. Run the search
optuna_search.fit(X_train, y_train)

In [ ]:
# 5. Inspect results
print("Best MAE  =", optuna_search.best_score_)
print("Best params:")
for k, v in optuna_search.best_params_.items():
    print(f"  • {k} = {v}")

# 6. Your final model
final_xgb_model = optuna_search.best_estimator_
final_xgb_model_params = optuna_search.best_params_
final_xgb_model_scores = optuna_search.best_score_

In [ ]:
val_final_xgb_model = final_xgb_model.predict(X_val)

# 7. Calculate regression metrics for validation set with Random Forest Regressor
final_val_mae_xgb = mean_absolute_error(y_val, val_final_xgb_model)
final_val_mse_xgb = mean_squared_error(y_val, val_final_xgb_model)
final_val_rmse_xgb = root_mean_squared_error(y_val, val_final_xgb_model)
final_val_r2_xgb = r2_score(y_val, val_final_xgb_model)

# 8. Display the metrics for Random Forest Regressor
print("Random Forest Regressor Metrics:")
print(f"Random Forest Validation MAE: {final_val_mae_xgb}")
print(f"Random Forest Validation MSE: {final_val_mse_xgb}")
print(f"Random Forest Validation RMSE: {final_val_rmse_xgb}")
print(f"Random Forest Validation R²: {final_val_r2_xgb}")

#### Light GBM Optuna Search CV

In [ ]:
# 1. grab your previously-found best params:
best_lgb_random_search_param = lgb_random_search.best_params_

# 2. build the “around-the-best” distributions:
lgb_param_distributions = make_tight_distributions(best_lgb_random_search_param,
                                               int_frac=0.2,     # ±20%
                                               float_frac=0.1,   # ±10%
                                               min_int_step=1)   # at least step=1

In [ ]:
lgb_param_distributions

In [ ]:
# 3. Optuna quick search
optuna_search = OptunaSearchCV(
    estimator=lgb_pipeline,
    param_distributions=lgb_param_distributions,
    cv=5,             
    n_trials=30,      
    scoring='neg_mean_absolute_error',
    random_state=42,
    n_jobs=-1,
    verbose=3,
)

# 4. Run the search
optuna_search.fit(X_train, y_train)

In [ ]:
# 5. Inspect results
print("Best MAE  =", optuna_search.best_score_)
print("Best params:")
for k, v in optuna_search.best_params_.items():
    print(f"  • {k} = {v!r}")

# 6. Your final model
final_lgb_model = optuna_search.best_estimator_
final_lgb_model_params = optuna_search.best_params_
final_lgb_model_scores = optuna_search.best_score_

In [ ]:
val_final_lgb_model = final_lgb_model.predict(X_val)

# 7. Calculate regression metrics for validation set with Random Forest Regressor
final_val_mae_lgb = mean_absolute_error(y_val, val_final_lgb_model)
final_val_mse_lgb = mean_squared_error(y_val, val_final_lgb_model)
final_val_rmse_lgb = root_mean_squared_error(y_val, val_final_lgb_model)
final_val_r2_lgb = r2_score(y_val, val_final_lgb_model)

# 8. Display the metrics for Random Forest Regressor
print("Random Forest Regressor Metrics:")
print(f"Random Forest Validation MAE: {final_val_mae_lgb}")
print(f"Random Forest Validation MSE: {final_val_mse_lgb}")
print(f"Random Forest Validation RMSE: {final_val_rmse_lgb}")
print(f"Random Forest Validation R²: {final_val_r2_lgb}")

#### Comparing Fine Tuned Scores

In [ ]:
# Print the best parameters and best score for Random Forest Regressor
print("Best parameters for Random Forest Regressor:", final_rf_model_params)
print("Best score for Random Forest Regressor (negative MSE):", final_rf_model_scores)
# Print the best parameters and best score for XGBoost Regressor
print("Best parameters for XGBoost Regressor:", final_xgb_model_params)
print("Best score for XGBoost Regressor (negative MSE):", final_xgb_model_scores)
# Print the best parameters and best score for LightGBM Regressor
print("Best parameters for LightGBM Regressor:", final_lgb_model_params)
print("Best score for LightGBM Regressor (negative MSE):", final_lgb_model_scores)


### Evaluation of Fine Tuned Models

**First, we apply the default decision tree as baseline for sanity check.**

In [ ]:
# Predict on the validation set with the default Decision Tree Regressor
y_val_pred_dt_default = dt_default_model.predict(X_val)
# Calculate regression metrics for validation set with the default Decision Tree Regressor
val_mae_dt_default = mean_absolute_error(y_val, y_val_pred_dt_default)
val_rmse_dt_default = root_mean_squared_error(y_val, y_val_pred_dt_default)
val_r2_dt_default = r2_score(y_val, y_val_pred_dt_default)

# Display the metrics for the default Decision Tree Regressor
print("Default Decision Tree Regressor Metrics:")
print(f"Default Decision Tree Validation MAE: {val_mae_dt_default}")
print(f"Default Decision Tree Validation RMSE: {val_rmse_dt_default}")
print(f"Default Decision Tree Validation R²: {val_r2_dt_default}")

**Next, we try Random Forest with default settings as it has a good score before fine tuning.**

In [ ]:
# Predict on the test set with the default Random Forest Regressor
y_val_pred_rf_default = rf_default_model.predict(X_val)
# Calculate regression metrics for test set with the default Random Forest Regressor
val_mae_rf_default = mean_absolute_error(y_val, y_val_pred_rf_default)
val_rmse_rf_default = root_mean_squared_error(y_val, y_val_pred_rf_default)
val_r2_rf_default = r2_score(y_val, y_val_pred_rf_default)

# Display the metrics for the default Random Forest Regressor
print("Default Random Forest Regressor Metrics:")
print(f"Default Random Forest Validation MAE: {val_mae_rf_default}")
print(f"Default Random Forest Validation RMSE: {val_rmse_rf_default}")
print(f"Default Random Forest Validation R²: {val_r2_rf_default}")

In [ ]:
# predict on the validation set with the best Random Forest Regressor
y_val_pred_rf_best = final_rf_model.predict(X_val)
# Calculate regression metrics for validation set with the best Random Forest Regressor
val_mae_rf_best = mean_absolute_error(y_val, y_val_pred_rf_best)
val_rmse_rf_best = root_mean_squared_error(y_val, y_val_pred_rf_best)
val_r2_rf_best = r2_score(y_val, y_val_pred_rf_best)    

# Display the metrics for the best Random Forest Regressor
print("Best Random Forest Regressor Metrics:")
print(f"Best Random Forest Validation MAE: {val_mae_rf_best}")
print(f"Best Random Forest Validation RMSE: {val_rmse_rf_best}")
print(f"Best Random Forest Validation R²: {val_r2_rf_best}")

In [ ]:
# predict on the validation set with the best xgboost Regressor
y_val_pred_xgb_best = final_xgb_model.predict(X_val)
# Calculate regression metrics for validation set with the best XGBoost Regressor
val_mae_xgb_best = mean_absolute_error(y_val, y_val_pred_xgb_best)
val_rmse_xgb_best = root_mean_squared_error(y_val, y_val_pred_xgb_best)
val_r2_xgb_best = r2_score(y_val, y_val_pred_xgb_best)  
# Display the metrics for the best XGBoost Regressor
print("Best XGBoost Regressor Metrics:")
print(f"Best XGBoost Validation MAE: {val_mae_xgb_best}")       
print(f"Best XGBoost Validation RMSE: {val_rmse_xgb_best}")
print(f"Best XGBoost Validation R²: {val_r2_xgb_best}") 


In [ ]:
# predict on the validation set with the best lightgbm Regressor
y_val_pred_lgb_best = final_lgb_model.predict(X_val)
# Calculate regression metrics for validation set with the best LightGBM Regressor
val_mae_lgb_best = mean_absolute_error(y_val, y_val_pred_lgb_best)
val_rmse_lgb_best = root_mean_squared_error(y_val, y_val_pred_lgb_best)
val_r2_lgb_best = r2_score(y_val, y_val_pred_lgb_best)
# Display the metrics for the best LightGBM Regressor
print("Best LightGBM Regressor Metrics:")
print(f"Best LightGBM Validation MAE: {val_mae_lgb_best}")
print(f"Best LightGBM Validation RMSE: {val_rmse_lgb_best}")
print(f"Best LightGBM Validation R²: {val_r2_lgb_best}")    


In [ ]:
# add ing metrics to dictionary for each model
metrics_rf_default = {
    "Model": "Random Forest (def)",
    "MAE":  val_mae_rf_default,
    "RMSE": val_rmse_rf_default,
    "R2":   val_r2_rf_default,
}

metrics_rf = {
    "Model": "Random Forest",
    "MAE":  val_mae_rf_best,
    "RMSE": val_rmse_rf_best,
    "R2":   val_r2_rf_best,
}

metrics_xgb = {
    "Model": "XGBoost",
    "MAE":  val_mae_xgb_best,
    "RMSE": val_rmse_xgb_best,
    "R2":   val_r2_xgb_best,
}

metrics_lgb = {
    "Model": "LightGBM",
    "MAE":  val_mae_lgb_best,
    "RMSE": val_rmse_lgb_best,
    "R2":   val_r2_lgb_best,
}

all_results = [metrics_rf_default, metrics_rf, metrics_xgb, metrics_lgb]

results_df = (
    pd.DataFrame(all_results)
      .set_index("Model")
      .round(4)          # nice, tidy formatting
)

In [ ]:
display(results_df)

**Our best model is XGBoost with MAE of $23,338 margin or errors.**

**All MAE are very close. In our experience, model with the best validation score may not do well in the test. Therefore, we will apply all the 4 models into the test set. We will deploy model with the best score.**



## Final Model Evaluation

In [ ]:
# Predict on the test set with the default Random Forest Regressor
y_test_pred_rf_default = rf_default_model.predict(X_test)
# Calculate regression metrics for test set with the default Random Forest Regressor
test_mae_rf_default = mean_absolute_error(y_test, y_test_pred_rf_default)
test_rmse_rf_default = root_mean_squared_error(y_test, y_test_pred_rf_default)
test_r2_rf_default = r2_score(y_test, y_test_pred_rf_default)

# Display the metrics for the default Random Forest Regressor
print("Default Random Forest Regressor Metrics:")
print(f"Default Random Forest Test MAE: {test_mae_rf_default}")
print(f"Default Random Forest Test RMSE: {test_rmse_rf_default}")
print(f"Default Random Forest Test R²: {test_r2_rf_default}")

In [ ]:
# Predict on the test set with the best Random Forest Regressor
y_test_pred_rf_best = final_rf_model.predict(X_test)
# Calculate regression metrics for test set with the best Random Forest Regressor
test_mae_rf_best = mean_absolute_error(y_test, y_test_pred_rf_best)
test_rmse_rf_best = root_mean_squared_error(y_test, y_test_pred_rf_best)
test_r2_rf_best = r2_score(y_test, y_test_pred_rf_best)

# Display the metrics for the best Random Forest Regressor
print("Best Random Forest Regressor Metrics:")
print(f"Best Random Forest Test MAE: {test_mae_rf_best}")
print(f"Best Random Forest Test RMSE: {test_rmse_rf_best}")
print(f"Best Random Forest Test R²: {test_r2_rf_best}")

In [ ]:
# Predict on the test set with the best XGBoost Regressor
y_test_pred_xgb_best = final_xgb_model.predict(X_test)
# Calculate regression metrics for test set with the best XGBoost Regressor
test_mae_xgb_best = mean_absolute_error(y_test, y_test_pred_xgb_best)
test_rmse_xgb_best = root_mean_squared_error(y_test, y_test_pred_xgb_best)
test_r2_xgb_best = r2_score(y_test, y_test_pred_xgb_best)

# Display the metrics for the best XGBoost Regressor
print("Best XGBoost Regressor Metrics:")
print(f"Best XGBoost Regressor Test MAE: {test_mae_xgb_best}")
print(f"Best XGBoost Regressor Test RMSE: {test_rmse_xgb_best}")
print(f"Best XGBoost Regressor Test R²: {test_r2_xgb_best}")

In [ ]:
# Predict on the test set with the best LightGBM Regressor
y_test_pred_lgb_best = final_lgb_model.predict(X_test)
# Calculate regression metrics for test set with the best LightGBM Regressor
test_mae_lgb_best = mean_absolute_error(y_test, y_test_pred_lgb_best)
test_rmse_lgb_best = root_mean_squared_error(y_test, y_test_pred_lgb_best)
test_r2_lgb_best = r2_score(y_test, y_test_pred_lgb_best)

# Display the metrics for the best LightGBM Regressor
print("Best LightGBM Regressor Metrics:")
print(f"Best LightGBM Regressor Test MAE: {test_mae_lgb_best}")
print(f"Best LightGBM Regressor Test RMSE: {test_rmse_lgb_best}")
print(f"Best LightGBM Regressor Test R²: {test_r2_lgb_best}")

**The best model is still XGBoost. MAE result is similar to validation test. This confirms that our test set is representative of the validation datasets.**

### Final Model Application

In [ ]:
# Best model from Optuna Search
best_model = final_xgb_model

# Predict on the test set with Ridge Regression
y_test_pred_best_model = best_model.predict(X_test)

# Calculate regression metrics for the test set for Ridge
test_mae_best = mean_absolute_error(y_test, y_test_pred_best_model)
test_rmse_best = root_mean_squared_error(y_test, y_test_pred_best_model)
test_r2_best = r2_score(y_test, y_test_pred_best_model)

print("Best Ridge Regression Model, Final Test Metrics:")
print(f"Final Test MAE: {test_mae_best}")
print(f"Final Test RMSE: {test_rmse_best}")
print(f"Final Test R²: {test_r2_best}")

## END